In [1]:
#svm is where we really step into the door of machine learning
#i personally think andrew ng did a shitty job in teaching this topic
#i strong recommend readers to go to the following website
# https://www.svm-tutorial.com/
#this site was created by a polish developer called Alexandre Kowalczyk
#he dedicated to teach svm from the very basic notation of vectors and lagrangian
#very friendly for beginners who forget everything about high school math
#and the free e-book he wrote is a must-read for more advanced techniques
#such as l1,l2 regularized soft margin, kernel, smo, multiclass classification

import cvxopt.solvers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import os
os.chdir('d:/python/data')

In [2]:
df=pd.read_csv('iris.csv')

In [3]:
#the classification has to be float instead of int
#this is requested by cvxopt
#for a binary classification
#the value should be either -1.0 or 1.0
df['y']=np.select([df['type']=='Iris-setosa', \
                   df['type']=='Iris-versicolor', \
                   df['type']=='Iris-virginica'],[-1.0,0.0,1.0])

In [4]:
#for the simplicity, let us make it a binary classification
df=df[df['y']!=0.0]

In [5]:
#for the simplicity, let us reduce the dimension of x to 2
temp=pd.concat([df[i] for i in df.columns if 'length' in i or 'width' in i],axis=1)
x=PCA(n_components=1).fit_transform(temp)

In [6]:
x=pd.Series([x[i].item() for i in range(len(x))])

In [7]:
y=df['y']

In [8]:
#train test split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [9]:
#crucial!!!!
#or we would get errors in the next step
x_test.reset_index(inplace=True,drop=True)
y_test.reset_index(inplace=True,drop=True)
x_train.reset_index(inplace=True,drop=True)
y_train.reset_index(inplace=True,drop=True)

In [10]:
#svm for binary classification
def binary_svm(x_train,x_test,y_train,y_test,kernel='linear',poly_constant=0.0,poly_power=1,gamma=5):

    #this is outer product matrix
    #which is the combination of all inner products
    #alternatively,we can write outer product in
    #np.mat([np.dot(y_train[i],y_train[j]) for j in y_train.index for i in y_train.index]).reshape(len(y_train),len(y_train))
    #or just np.mat(y_train).T*np.mat(y_train)
    y_product=np.outer(y_train,y_train)
    
    #using different kernels to map inner product to a higher dimension space
    #there are only three kernels here, which are linear, polynomial, gaussian
    if kernel=='linear':
        x_product=np.outer(x_train,x_train)
    elif kernel=='polynomial':
        temp=np.outer(x_train,x_train)
        x_product=np.apply_along_axis(lambda x:(x+poly_constant)**poly_power,0,temp.ravel()).reshape(temp.shape)
    else:
        #gaussian/rbf kernel
        #map to infinitive dimension space
        temp=np.mat([i-j for j in x_train for i in x_train]).reshape(len(x_train),len(x_train))
        x_product=np.apply_along_axis(lambda x:np.exp(-1*gamma*(np.linalg.norm(x))**2),0,temp.ravel()).reshape(temp.shape)
    
    #plz refer to the following link for how to solve wolfe dual problem in cvxopt
    # http://cvxopt.org/userguide/coneprog.html#quadratic-programming
    P=cvxopt.matrix(x_product*y_product)
    q=cvxopt.matrix(-1*np.ones(len(x_train)))
    G=cvxopt.matrix(np.diag(-1 * np.ones(len(x_train))))
    h=cvxopt.matrix(np.zeros(len(x_train)))
    A=cvxopt.matrix(y_train,(1,len(x_train)))
    b=cvxopt.matrix(0.0)

    solution=cvxopt.solvers.qp(P, q, G, h, A, b)
    alpha=pd.Series(solution['x'])
    w=np.sum(alpha*y_train*x_train)

    #here i am using prof andrew ng's way
    #alternatively, we can do a normal average of all value b
    #b=np.mean(y_train-w*x_train)
    b=-(min(x_train[y_train==1.0]*w)+max(x_train[y_train==-1.0]*w))/2

    print('\ntrain accuracy: %s'%(len(y_train[np.sign(np.multiply(w,x_train)+b)==y_train])/len(y_train)*100)+'%')
    print('\ntest accuracy: %s'%(len(y_test[np.sign(np.multiply(w,x_test)+b)==y_test])/len(y_test)*100)+'%')
    print('\nparameters w: %s'%(w))
    print('\nparameters b: %s'%(b))

In [11]:
binary_svm(x_train,x_test,y_train,y_test,kernel='gaussian')

     pcost       dcost       gap    pres   dres
 0: -2.8386e+00 -8.7437e+00  6e+00  3e-16  1e+00
 1: -2.9601e+00 -3.6279e+00  7e-01  2e-16  2e-01
 2: -3.0560e+00 -3.1621e+00  1e-01  2e-16  2e-02
 3: -3.0967e+00 -3.1123e+00  2e-02  1e-16  1e-16
 4: -3.1025e+00 -3.1055e+00  3e-03  2e-16  1e-16
 5: -3.1034e+00 -3.1049e+00  1e-03  3e-16  1e-16
 6: -3.1041e+00 -3.1044e+00  3e-04  4e-16  1e-16
 7: -3.1043e+00 -3.1043e+00  6e-05  4e-16  1e-16
 8: -3.1043e+00 -3.1043e+00  8e-06  7e-16  2e-16
 9: -3.1043e+00 -3.1043e+00  2e-07  4e-16  2e-16
Optimal solution found.

train accuracy: 100.0%

test accuracy: 100.0%

parameters w: 15.124567872650465

parameters b: 8.641313505812779


In [12]:
#using official sklearn package with the same parameters
def skl_binary_svm(x_train,x_test,y_train,y_test,kernel='rbf',gamma=5,**kwargs):
    
    m=SVC(kernel=kernel,gamma=gamma, \
          **kwargs).fit(np.array(x_train).reshape(-1, 1), \
                        np.array(y_train).ravel())
    
    train=m.score(np.array(x_train).reshape(-1, 1), \
                  np.array(y_train).ravel())*100
    test=m.score(np.array(x_test).reshape(-1, 1), \
                 np.array(y_test).ravel())*100
    
    print('\ntrain accuracy: %s'%(train)+'%')
    print('\ntest accuracy: %s'%(test)+'%')

In [13]:
skl_binary_svm(x_train,x_test,y_train,y_test)


train accuracy: 100.0%

test accuracy: 100.0%
